<a href="https://colab.research.google.com/github/qAp/soil/blob/develop/nbs/00_points_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sampling images

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) 

import pandas as pd
import ee
import geemap    

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=T91Ht3XRuj4XwuVsXyGKmwIm3quwyq4sRMfL-x94hdM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6WoS0LB_LPiNrrwaTUtzt_FY00HK-dKAWiyiOCmrb0Lp_Olr3hXhQ

Successfully saved authorization token.


## Soil samples

In [3]:
wise3 = ee.FeatureCollection("users/bingosaucer/orgc_stock_ISRIC_WISE_clip_gee")

In [4]:
wise3.getInfo()['features'][0]

{'geometry': {'coordinates': [-155.1005552139237, 19.744168172085583],
  'type': 'Point'},
 'id': '0000000000000000011a',
 'properties': {'bot_cm': 100,
  'datetime': 0,
  'mineral_fraction': 0.23000000417232513,
  'organic_fraction': 0.7699999809265137,
  'orgc_stock_mineral': 11.028960668150008,
  'orgc_stock_organic': 7.822259886666,
  'profile_id': 'US0004',
  'top_cm': 0},
 'type': 'Feature'}

## Merge images in a collection to an image

In [5]:
def collection2image_stackbands(collection):
    '''
    Merge images in an image collection to a single image.  Each
    image in the collection becomes a band in the output image.
    
    Args:
        collection (ee.ImageCollection): Image collection.
    '''
    img = collection.iterate(lambda curr, prev: ee.Image(prev).addBands(curr), ee.Image(1))
    img = ee.Image(img)
    ibands = ee.List.sequence(1, img.bandNames().size().subtract(1))
    return img.select(ibands)


In [6]:
landcovers = ee.ImageCollection('users/bingosaucer/ConsensusLandCover_Human_Development_Percentage')

In [7]:
def get_earthenv_landcover_classes():
    '''
    https://www.earthenv.org/landcover
    Returns: Global 1-km Consensus Land Cover classes
    '''
    return ['Evergreen/Deciduous Needleleaf Trees', 'Evergreen Broadleaf Trees', 'Deciduous Broadleaf Trees', 
            'Mixed/Other Trees', 'Shrubs', 'Herbaceous Vegetation', 'Cultivated and Managed Vegetation',
            'Regularly Flooded Vegetation', 'Urban/Built-up', 'Snow/Ice', 'Barren', 'Open Water']

In [8]:
landcovers_img = collection2image_stackbands(landcovers)

landcovers_img = landcovers_img.rename(get_earthenv_landcover_classes())
landcovers_img.bandNames().getInfo()

['Evergreen/Deciduous Needleleaf Trees',
 'Evergreen Broadleaf Trees',
 'Deciduous Broadleaf Trees',
 'Mixed/Other Trees',
 'Shrubs',
 'Herbaceous Vegetation',
 'Cultivated and Managed Vegetation',
 'Regularly Flooded Vegetation',
 'Urban/Built-up',
 'Snow/Ice',
 'Barren',
 'Open Water']

In [9]:
Map = geemap.Map(center=(40, -100), zoom=3)
for landcover in get_earthenv_landcover_classes():
    Map.addLayer(landcovers_img.select(landcover), 
                {'palette': ['#ffffcc', '#e31a1c', '#bd0026', '#800026'], 'min':0, 'max':100}, 
                landcover, True, 0.3)
Map.addLayer(wise3, {}, "WISE3")
Map

## Sampling image at points

> images + points $\rightarrow$ images' values at points

Use `ee.Image.reduceRegions` to sample the above image at a selection of points.  The result is a `ee.FeatureCollection`, where each feature is one of the selected points.  Each point has a `properties` dict, and each property inside this is the value a band of an image from the original image collection.

In [10]:
result = landcovers_img.reduceRegions(wise3, ee.Reducer.mean())

In [11]:
result.getInfo()['features'][890]

{'geometry': {'coordinates': [116.72083165545814, -0.6958293420870688],
  'type': 'Point'},
 'id': '00000000000000000415',
 'properties': {'Barren': 0,
  'Cultivated and Managed Vegetation': 0,
  'Deciduous Broadleaf Trees': 0,
  'Evergreen Broadleaf Trees': 0,
  'Evergreen/Deciduous Needleleaf Trees': 0,
  'Herbaceous Vegetation': 0,
  'Mixed/Other Trees': 0,
  'Open Water': 0,
  'Regularly Flooded Vegetation': 0,
  'Shrubs': 86,
  'Snow/Ice': 14,
  'Urban/Built-up': 0,
  'bot_cm': 100,
  'datetime': 1988,
  'mineral_fraction': 1,
  'organic_fraction': 0,
  'orgc_stock_mineral': 8.47130012512207,
  'orgc_stock_organic': 0,
  'profile_id': 'ID0090',
  'top_cm': 0},
 'type': 'Feature'}

## Convert feature collection to dataframe

In [12]:
def add_coords_to_feature_properties(feature):
    properties = feature['properties']
    lon, lat = feature['geometry']['coordinates']
    properties.update({'longitude':lon, 'latitude':lat})
    return properties

In [13]:
add_coords_to_feature_properties(result.getInfo()['features'][890])

{'Barren': 0,
 'Cultivated and Managed Vegetation': 0,
 'Deciduous Broadleaf Trees': 0,
 'Evergreen Broadleaf Trees': 0,
 'Evergreen/Deciduous Needleleaf Trees': 0,
 'Herbaceous Vegetation': 0,
 'Mixed/Other Trees': 0,
 'Open Water': 0,
 'Regularly Flooded Vegetation': 0,
 'Shrubs': 86,
 'Snow/Ice': 14,
 'Urban/Built-up': 0,
 'bot_cm': 100,
 'datetime': 1988,
 'latitude': -0.6958293420870688,
 'longitude': 116.72083165545814,
 'mineral_fraction': 1,
 'organic_fraction': 0,
 'orgc_stock_mineral': 8.47130012512207,
 'orgc_stock_organic': 0,
 'profile_id': 'ID0090',
 'top_cm': 0}

In [14]:
data_dicts = (add_coords_to_feature_properties(feature) for feature in result.getInfo()['features'])

In [15]:
pd.DataFrame.from_records(data_dicts)

,Barren,Cultivated and Managed Vegetation,Deciduous Broadleaf Trees,Evergreen Broadleaf Trees,Evergreen/Deciduous Needleleaf Trees,Herbaceous Vegetation,Mixed/Other Trees,Open Water,Regularly Flooded Vegetation,Shrubs,Snow/Ice,Urban/Built-up,bot_cm,datetime,mineral_fraction,organic_fraction,orgc_stock_mineral,orgc_stock_organic,profile_id,top_cm,longitude,latitude
0,0,10,0,0,0,12,0,0,0,23,19,5,100,0,0.23,0.77,11.028961,7.822260,US0004,0,-155.100555,19.744168
1,0,3,9,0,0,0,0,0,0,0,71,17,100,0,0.20,0.80,68.639999,41.761501,DE0012,0,8.033329,53.166669
2,0,0,9,0,0,14,0,0,0,0,77,0,100,0,0.27,0.73,61.538399,95.802002,DE0013,0,8.216670,52.483330
3,0,49,0,0,17,0,17,0,0,0,17,0,100,1936,0.10,0.90,1.170000,15.498000,RU0010,0,31.499999,58.099719
4,0,22,0,0,0,0,0,0,0,33,45,0,100,1976,0.20,0.80,13.420000,76.601200,CO0062,0,-73.866668,5.583330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,0,0,0,0,0,0,100,0,0,0,0,0,100,1982,1.00,0.00,19.089100,0.000000,PA0010,0,-79.500002,8.950000
1473,0,0,0,0,0,0,100,0,0,0,0,0,100,1982,1.00,0.00,12.583760,0.000000,PA0011,0,-79.500002,8.950000
1474,0,0,0,0,0,0,100,0,0,0,0,0,100,1982,1.00,0.00,7.289500,0.000000,PA0012,0,-79.500002,8.950000
1475,0,0,0,0,0,0,100,0,0,0,0,0,100,1982,1.00,0.00,8.554920,0.000000,PA0013,0,-79.500002,8.950000


# Reference

- https://mygeoblog.com/2019/07/17/sample-an-image-collection/